In [1]:
import numpy as np
import tensorflow as tf
import word_preprocessing as wp

[nltk_data] Downloading package punkt to /home/kaushik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
ml_classes = ['location', 'professor', 'time']
processor  =  wp.Word_Processor('../grammar/')
feature_set,class_set,lexicon_size = processor.words_to_vectors()


In [3]:
lstm_units = 128
batch_size = 10
lstm_layers = 1
learning_rate = 0.01

def split_sets(data, classes ,split_point = 1):
    split_length  = int(data.shape[0] * split_point)
    training_features = data[:split_length]
    training_ops = classes[:split_length]
    testing_features = data[split_length:]
    testing_ops = classes[split_length:]
    return training_features,training_ops,testing_features,testing_ops

train_x,train_y,test_x,test_y = split_sets(feature_set, class_set)


In [4]:
outputs = tf.placeholder(tf.int64 , [batch_size, 1])
y  = tf.one_hot(indices=outputs,depth=len(ml_classes))
inputs = tf.placeholder(tf.float64, [batch_size, lexicon_size, 1], name="inputs")


def create_lstm_network(batch_size,inputs):
    def create_lstm_cell(units):
        lstm  = tf.contrib.rnn.BasicLSTMCell(units)
        return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=1)

    def create_stacked_lstm(layers, units):
        return tf.contrib.rnn.MultiRNNCell([create_lstm_cell(units) for _ in range(layers)])
    
    cell =  create_lstm_cell(lstm_units)
    inits = cell.zero_state(batch_size,tf.float64)
    lstm_outputs,states =  tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float64, initial_state=inits)
    predictions = tf.contrib.layers.fully_connected(lstm_outputs[:, -1], num_outputs= len(ml_classes), activation_fn=tf.nn.relu)
    return predictions

In [5]:
pred = create_lstm_network(batch_size,inputs)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits=pred))
tf.summary.scalar('cost', cost)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
tf.summary.histogram('predictions', pred)
merged = tf.summary.merge_all()

In [6]:
def get_batches(x, y, batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield np.array(x[ii:ii+batch_size]), np.array(y[ii:ii+batch_size])

In [7]:
epochs = 25
from tensorflow.python import debug as tf_debug

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
   # sess = tf_debug.LocalCLIDebugWrapperSession(sess)
   # sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)
    train_writer = tf.summary.FileWriter('./logs/lstm-chatbot/train', sess.graph)
    test_writer = tf.summary.FileWriter('./logs/lstm-chatbot/test', sess.graph)
    iteration = 1
    for epoch in range(epochs):
        #state = sess.run(initial_state)
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            x  =  np.reshape(x, newshape = [batch_size ,lexicon_size, 1])
            y =  np.reshape(y, newshape=[batch_size,1])
            feed = {inputs: x,outputs : y}
            #sess.run(outputs, feed_dict=feed)
            #print(outputs.eval(feed_dict=feed))
            pre,summary, loss, _ = sess.run([pred,merged, cost, optimizer], feed_dict=feed)
            train_writer.add_summary(summary, iteration)
        
            if iteration%100 == 0:
                print("Epoch: {}/{}".format(epoch, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))
                   
            iteration +=1
     
            
    saver.save(sess, "checkpoints/lstm_chatbot.ckpt")
 
    
    
    

Epoch: 0/25 Iteration: 100 Train loss: 0.933
Epoch: 1/25 Iteration: 200 Train loss: 0.833
Epoch: 2/25 Iteration: 300 Train loss: 0.457
Epoch: 2/25 Iteration: 400 Train loss: 1.161
Epoch: 3/25 Iteration: 500 Train loss: 0.321
Epoch: 4/25 Iteration: 600 Train loss: 0.480
Epoch: 4/25 Iteration: 700 Train loss: 0.312
Epoch: 5/25 Iteration: 800 Train loss: 0.144
Epoch: 6/25 Iteration: 900 Train loss: 0.242
Epoch: 6/25 Iteration: 1000 Train loss: 0.085
Epoch: 7/25 Iteration: 1100 Train loss: 0.181
Epoch: 8/25 Iteration: 1200 Train loss: 0.460
Epoch: 8/25 Iteration: 1300 Train loss: 0.119
Epoch: 9/25 Iteration: 1400 Train loss: 0.156
Epoch: 10/25 Iteration: 1500 Train loss: 0.151
Epoch: 11/25 Iteration: 1600 Train loss: 0.295
Epoch: 11/25 Iteration: 1700 Train loss: 0.062
Epoch: 12/25 Iteration: 1800 Train loss: 0.439
Epoch: 13/25 Iteration: 1900 Train loss: 0.248
Epoch: 13/25 Iteration: 2000 Train loss: 0.084
Epoch: 14/25 Iteration: 2100 Train loss: 0.193
Epoch: 15/25 Iteration: 2200 Train l

In [27]:
test_batch = 1
tf.reset_default_graph()
test_input = tf.placeholder(tf.float64, [test_batch, lexicon_size, 1], name="test_inputs")
test_prediction = create_lstm_network(test_batch, test_input)
with tf.Session() as sess:
    #test_prediction = create_lstm_network(test_batch, test_input)
    sess.run(tf.global_variables_initializer())
    saver =  tf.train.Saver()
    # = tf.train.import_meta_graph('checkpoints/lstm_chatbot.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint('checkpoints/'))
    tr_data = processor.vectorize_input("us president")
    tr_data = np.reshape(tr_data, newshape=[test_batch, lexicon_size , 1])
    pred_class = sess.run(test_prediction, feed_dict={test_input:tr_data})
    print(pred_class)
    for var in tf.trainable_variables():
        print(var.name)
            
      









    

INFO:tensorflow:Restoring parameters from checkpoints/lstm_chatbot.ckpt
[[ 0.          0.          5.34603268]]
rnn/basic_lstm_cell/kernel:0
rnn/basic_lstm_cell/bias:0
fully_connected/weights:0
fully_connected/biases:0
